In [ ]:

from delta.tables import *
from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark.sql.utils import *
from pyspark.sql.window import *

In [ ]:
spark = SparkSession.builder.getOrCreate()

In [ ]:
##READ FILES

manual_columns = [
    "ano", "cnpj", "cnpj_scp", "forma_tributacao", "quantidade_escrituracoes"
]

df_regime_tributario = spark.read.format("csv").option("header", "false").option("sep", ";").load(
    './Regime_Tributario/*.csv')
df_regime_tributario = df_regime_tributario.toDF(*manual_columns)
df_regime_tributario.createOrReplaceTempView("regime_tributario")

In [ ]:
df_final = spark.sql('''
    SELECT
        ano,
        REPLACE(REPLACE(REPLACE(cnpj, '.', ''), '/', ''), '-', '') AS cnpj,
        cnpj_scp,
        forma_tributacao,
        quantidade_escrituracoes        
    FROM
        regime_tributario
''')
df_final.show()

In [ ]:
df_final.coalesce(1).write.mode('overwrite').parquet('./Regime_Tributario/arquivo_parquet')